In [ ]:
our proget

#import libraries 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


#import data 
df = pd.read_csv(r'C:/Users/eleon/Desktop/ai.project/customer_segmentation.csv')
print(df)



# Taking a look at the data structure
#df.describe( ) on the data to se the statistic of each variable 

df.describe() 
df.head()
df.info()
df.describe()
df.shape


#our second step is creating the test set 
#to create the test set we do:
    
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

# Data preparation
df.duplicated().sum()

def fix_duplicates(Dataframe):
  dataframeHeaders = list(Dataframe)
  del dataframeHeaders[0]
  Dataframe.drop_duplicates(subset = dataframeHeaders, keep = 'first', inplace = True)
  Dataframe.reset_index(drop = True, inplace = True)
  return Dataframe

df = fix_duplicates(df)

df.isnull().sum()

df['index_col'] = df.index

#df.plot.scatter(x='index_col',y='payment_installments')
#df.plot.scatter(x='index_col',y='payment_value')
#df.plot.scatter(x='index_col',y='price')
#df.plot.scatter(x='index_col',y='freight_value')

#sns.countplot(y='customer_state', data=df )

#sns.pairplot(df)

plt.figure()
df.plot.scatter(x='index_col',y='payment_installments')
plt.figure()
df.plot.scatter(x='index_col',y='payment_value')
plt.figure()
df.plot.scatter(x='index_col',y='price')
plt.figure()
df.plot.scatter(x='index_col',y='freight_value')
plt.figure()
sns.countplot(y='customer_state', data=df )






#we want to identify differ pattersn along the data
#and create different gruop according to does pattern (clusters)

#To get some insights we plot and visualize the data set

#See the distribution of city to recognize different distributions
#sns.countplot(x='customer_city', data=df );
##plt.title('Distribution_customer_citys'); #very strange result 

#sns.countplot(x='customer_state', data=df );
#plt.title('Distribution_customer_state');
#plt.show()
#sns.countplot(x="custumer_city", hue="custumer_state", data=df)
plt.figure()
sns.heatmap(df.corr(), annot=True)

#we will now explore the traning set to get some insights 
#scatterplots:

#thinghs we noticed --> we have to colums that are the same (p.name) , and we also product name length is not useful 


 #we ca usa dataframe.corr(), to find the correlation between numeric variables only.   
plt.figure()
corr_matrix = df.corr()
plt.figure()
from pandas.plotting import scatter_matrix
attributes = [ "price", "freight_value"]
scatter_matrix(df[attributes], figsize=(12, 8))

print(corr_matrix)




sns.heatmap(df.corr(), annot=True)
